In [6]:
%pip install selenium
%pip install webdriver_manager


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import csv

In [8]:
start_year = 2016
end_year = 2025
count = 100

urls = list()

for year in range(start_year, end_year + 1):

    url = f"https://www.imdb.com/search/title/?title_type=feature&release_date={year}-01-01,{year}-12-31&count={count}&sort=boxoffice_gross_us,desc"
    urls.append(url)

In [9]:
print(*urls, sep='\n')

https://www.imdb.com/search/title/?title_type=feature&release_date=2016-01-01,2016-12-31&count=100&sort=boxoffice_gross_us,desc
https://www.imdb.com/search/title/?title_type=feature&release_date=2017-01-01,2017-12-31&count=100&sort=boxoffice_gross_us,desc
https://www.imdb.com/search/title/?title_type=feature&release_date=2018-01-01,2018-12-31&count=100&sort=boxoffice_gross_us,desc
https://www.imdb.com/search/title/?title_type=feature&release_date=2019-01-01,2019-12-31&count=100&sort=boxoffice_gross_us,desc
https://www.imdb.com/search/title/?title_type=feature&release_date=2020-01-01,2020-12-31&count=100&sort=boxoffice_gross_us,desc
https://www.imdb.com/search/title/?title_type=feature&release_date=2021-01-01,2021-12-31&count=100&sort=boxoffice_gross_us,desc
https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&count=100&sort=boxoffice_gross_us,desc
https://www.imdb.com/search/title/?title_type=feature&release_date=2023-01-01,2023-12-31&count=100&sort=

In [10]:
with open("test.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f, delimiter=';')
    csv_header = ['year', 'place', 'title', 'rating', 'min',
                    'director', 'genre', 'release_date', 'origin_country',
                    'company', 'budget', 'gross_us', 'gross_world', 'open_week']
    
    writer.writerow(csv_header)

In [11]:
def get_info_from_url(html, headers, data):
    
    year = get_year(url)
    
    movie_url_start = "https://www.imdb.com"
    # response = requests.get(url, headers=headers)
    # print(response)
    soup = BeautifulSoup(html, "html.parser")
    movies = soup.select(".ipc-metadata-list-summary-item")
    print(len(movies))
    
    for movie in movies:
        movie_data = list()
        movie_data.append(year)
        link = movie.select(".ipc-title-link-wrapper")[0]
        place = link.text.split('.')[0].strip()
        movie_data.append(place)

        movie_url = movie_url_start + link.get('href')   
        get_info_from_movie_url(movie_url, headers, movie_data)
        data.append(movie_data)
        print(movie_data[0], movie_data[1])


def get_info_from_movie_url(url, headers, movie_data):

    response = requests.get(url, headers=headers)
    # print(response)
    soup = BeautifulSoup(response.text, "html.parser")
    title = get_text_from_class(soup, ".hero__primary-text")
    # director = get_text_from_class(soup, "sc-af040695-3 dUiZpV")
   
    rating_obj = soup.find(attrs={"data-testid": "hero-rating-bar__aggregate-rating__score"})

    if type(rating_obj) != type(None):
        rating = rating_obj.find("span")
        if type(rating) != type(None):
            rating = rating.text
        else:
            rating = ''
    else:
        rating = ''

    genre_obj = soup.find(class_="ipc-chip-list__scroller")
    genre_list = list()
    if type(genre_obj) != type(None):
        for genre in genre_obj.find_all(class_="ipc-chip__text"):
            genre_list.append(genre.text)
    
    director = soup.find(class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
    if type(director) != type(None):
        director = director.text    
    else:
        director = ''


    details_section = soup.find(attrs={"data-testid": "title-details-section"})

    if type(details_section) != type(None):
        release_date_section = details_section.find(attrs={"data-testid": "title-details-releasedate"})
        if type(release_date_section) != type(None):
            release_date = release_date_section.find(class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link").text
        else:
            release_date = ''
        
        OriginCountry_section = details_section.find(attrs={"data-testid": "title-details-origin"})
        if type(OriginCountry_section) != type(None):
            OriginCountry = OriginCountry_section.find(class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link").text
        else:
            OriginCountry = ''

        Company_section = details_section.find(attrs={"data-testid": "title-details-companies"})
        if type(Company_section) != type(None):
            Company = Company_section.find(class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link").text
        else:
            Company = ''
    else:
        release_date = ''
        OriginCountry = ''
        Company = ''

    BoxOffice_section = soup.find(attrs={"data-testid": "title-boxoffice-section"})

    if type(BoxOffice_section) != type(None):
        budget_obj = BoxOffice_section.find(attrs={"data-testid": "title-boxoffice-budget"})
        if type(budget_obj) != type(None):
            budget = budget_obj.find(class_="ipc-metadata-list-item__list-content-item ipc-btn--not-interactable").text
        else:
            budget = ""

        grossdomestic_obj = BoxOffice_section.find(attrs={"data-testid": "title-boxoffice-grossdomestic"})
        if type(grossdomestic_obj) != type(None):
            grossdomestic = grossdomestic_obj.find(class_="ipc-metadata-list-item__list-content-item ipc-btn--not-interactable").text
        else:
            grossdomestic = ""

        openingweekenddomestic_obj = BoxOffice_section.find(attrs={"data-testid": "title-boxoffice-openingweekenddomestic"})
        if type(openingweekenddomestic_obj) != type(None):
            openingweekenddomestic = openingweekenddomestic_obj.find(class_="ipc-metadata-list-item__list-content-item ipc-btn--not-interactable").text
        else:
            openingweekenddomestic = ""

        cumulativeworldwidegross_obj = BoxOffice_section.find(attrs={"data-testid": "title-boxoffice-cumulativeworldwidegross"})
        if type(cumulativeworldwidegross_obj) != type(None):
            cumulativeworldwidegross = cumulativeworldwidegross_obj.find(class_="ipc-metadata-list-item__list-content-item ipc-btn--not-interactable").text
        else:
            cumulativeworldwidegross = ""
    else:
        budget = ""
        grossdomestic = ""
        openingweekenddomestic = ""
        cumulativeworldwidegross = ""

    runtime_section = soup.find(attrs={"data-testid": "title-techspec_runtime"})    
    if type(runtime_section) != type(None):
        runtime_min = runtime_section.find(class_="ipc-metadata-list-item__list-content-item--subText")
        if type(runtime_min) != type(None):
            runtime_min = runtime_min.text
        else:
            runtime_min = ''    
    else:
        runtime_min = ''

    movie_data.append(title.strip())
    movie_data.append(rating.strip())
    movie_data.append(runtime_min.strip())
    movie_data.append(director.strip())
    movie_data.append(genre_list)
    movie_data.append(release_date.strip())
    movie_data.append(OriginCountry.strip())
    movie_data.append(Company.strip())
    movie_data.append(budget.strip())
    movie_data.append(grossdomestic.strip())
    movie_data.append(cumulativeworldwidegross.strip())
    movie_data.append(openingweekenddomestic.strip())

    
    
def get_text_from_class(soup, soup_class):
    a_row = soup.select(soup_class)
    if len(a_row) > 0:
        a_row = a_row[0]
        text = a_row.text
    else:
        text = ""
    return text

def get_year(url):
    key = 'release_date='
    start = url.find(key)

    if start != -1:
        year = url[start + len(key): start + len(key) + 4]
    else:
        year = '0000'
    
    return year

In [12]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/117.0",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.imdb.com/",
}

for url in urls:
    data = list()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    # time.sleep(2)
    html = driver.page_source
    driver.quit()
    get_info_from_url(html, headers, data)
    print('data', data)

    with open("test.csv", "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerows(data)


WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x58800b9fb4e3 <unknown>
#1 0x58800b72ac76 <unknown>
#2 0x58800b751757 <unknown>
#3 0x58800b750029 <unknown>
#4 0x58800b78eccc <unknown>
#5 0x58800b78e47f <unknown>
#6 0x58800b785de3 <unknown>
#7 0x58800b75b2dd <unknown>
#8 0x58800b75c34e <unknown>
#9 0x58800b9bb3e4 <unknown>
#10 0x58800b9bf3d7 <unknown>
#11 0x58800b9c9b20 <unknown>
#12 0x58800b9c0023 <unknown>
#13 0x58800b98e1aa <unknown>
#14 0x58800b9e46b8 <unknown>
#15 0x58800b9e4847 <unknown>
#16 0x58800b9f4243 <unknown>
#17 0x7b6bf259eac3 <unknown>


In [ ]:
df = pd.read_csv("../data/raw/imdb_data.csv", delimiter = ';')
df

,year,place,title,rating,min,director,genre,release_date,origin_country,company,budget,gross_us,gross_world,open_week
0,1975,1,Jaws,8.1,(124 min),Steven Spielberg,"['Disaster', 'Monster Horror', 'Sea Adventure'...","June 20, 1975 (United States)",United States,Zanuck/Brown Productions,"$7,000,000 (estimated)","$280,083,300","$490,736,300","$7,061,513"
1,1975,2,The Rocky Horror Picture Show,7.4,(100 min),Jim Sharman,"['B-Horror', 'Dark Comedy', 'Parody', 'Raunchy...","September 29, 1975 (United States)",United Kingdom,Twentieth Century Fox,"$1,200,000 (estimated)","$113,028,197","$116,574,698","$135,000"
2,1975,3,One Flew Over the Cuckoo's Nest,8.6,(133 min),Milos Forman,"['Dark Comedy', 'Medical Drama', 'Psychologica...","November 21, 1975 (United States)",United States,Fantasy Films,"$3,000,000 (estimated)","$108,981,275","$109,129,807",NaN
3,1975,4,Dog Day Afternoon,8.0,(125 min),Sidney Lumet,"['Dark Comedy', 'Heist', 'True Crime', 'Biogra...","December 25, 1975 (United States)",United States,Warner Bros.,"$1,800,000 (estimated)","$50,000,000","$50,009,617",NaN
4,1975,5,Shampoo,6.4,(110 min),Hal Ashby,"['Satire', 'Comedy', 'Drama']","March 13, 1975 (United States)",United States,Persky-Bright / Vista,"$4,000,000 (estimated)","$49,407,734","$49,407,734",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4870,2025,96,Eternity,7.2,(114 min),David Freyne,"['Feel-Good Romance', 'Romantic Comedy', 'Supe...","November 26, 2025 (United States)",United States,A24,NaN,"$5,236,587","$5,433,602","$3,169,780"
4871,2025,97,Hurry Up Tomorrow,4.4,(105 min),Trey Edward Shults,"['Psychological Thriller', 'Thriller']","May 16, 2025 (United States)",United States,Lionsgate,"$15,000,000 (estimated)","$5,215,357","$7,763,862","$3,312,692"
4872,2025,98,Light of the World,6.5,(91 min),Tom Bancroft,"['Hand-Drawn Animation', 'Animation', 'Drama',...","September 5, 2025 (United States)",United States,The Salvation Poem Project,"$20,000,000 (estimated)","$5,213,377","$5,604,948","$2,401,374"
4873,2025,99,Chhaava,7.3,(161 min),Laxman Utekar,"['Hindi', 'Action Epic', 'Costume Drama', 'Epi...","February 14, 2025 (India)",India,Maddock Films,NaN,"$4,800,000","$8,219,603","$1,790,000"
